In [3]:
import matplotlib.pyplot as plt 
import pandas as pd
from matplotlib.patches import Rectangle, FancyBboxPatch
import matplotlib.font_manager

In [236]:
def violations_boxplot(slo):
    def column_rename(columns):
        lookup = {'random': 'Random', 'oracle': 'Heatmap', 'whisker': 'Whisker', 'delphi': '', 'delphi3': 'N', \
                  'p': ('Pred.', 'Pred.', ''), 'r': ('Real', 'Oracle', ''), 'a': ('Attackers', 'Attackers', ''), '2': ('Conserv.', ''), '3': ('Moderate','')}
        renamed_columns = dict()
        for column in columns:
            tokens = column.split('_')
            new_name =  f"{lookup[tokens[3]][int('delphi' not in tokens[1] or 'a' in tokens[2])]} " + \
                        f"{lookup[tokens[2]][int('delphi' not in tokens[1]) + int(tokens[1] == 'random' or tokens[1] == 'whisker')]} " + \
                        f"{lookup[tokens[1]]}"
            renamed_columns[column] = new_name.strip()
        return renamed_columns

    with open(f"boxplot-{slo}.csv", 'r') as violations_file:
        df = pd.read_csv(violations_file, delimiter = ',')
    df = df.rename(columns = column_rename(df.columns))
    bp = df.boxplot(figsize = (26,20), return_type = 'both', rot = 90, patch_artist=True)

    edgecs = ['#227522'] * len(set(df.columns)) + ['#757522'] * len(set(df.columns)) + ['#752222'] * len(set(df.columns))
    colors = ['#A5DCA5'] * len(set(df.columns)) + ['#DCDCA5'] * len(set(df.columns)) + ['#DCA5A5'] * len(set(df.columns))
    linewidth = 2 
    fontsize = 50
    fname = "Times New Roman"
    (y_bottom, y_top) = (0.0, 30.0)
    if str(slo) == '1.1': plt.ylim(bottom = y_bottom)
    elif str(slo) == '1.2': plt.ylim(bottom = y_bottom, top = y_top - 3.0)
    elif str(slo) == 'all': plt.ylim(bottom = y_bottom)
    #bp[0].set_xticklabels(labels)
    #plt.yticks(np.arange(0.0, 22.6 if sla < 1.3 else 27.6, 2.5))
    for (patch, color, edgec) in zip(bp[1]['boxes'], colors, edgecs):
        patch.set(color = edgec, linewidth = linewidth)
        patch.set_facecolor(color)
    for (i, whisker) in enumerate(bp[1]['whiskers']): whisker.set(color = edgecs[i // 2], linewidth = linewidth)
    for (i, cap) in enumerate(bp[1]['caps']):
        cap.set(color=edgecs[i // 2], xdata=cap.get_xdata() - (-0.005,+0.005), linewidth = linewidth)
    for (median, color) in zip(bp[1]['medians'], edgecs): median.set(color=color, linewidth = linewidth)
    for (flier, color, edgec) in zip(bp[1]['fliers'], colors, edgecs):
        flier.set(marker = 'o', markerfacecolor = color, markeredgecolor = edgec, markersize = 8)
    for tick in bp[0].xaxis.get_major_ticks(): 
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontname(fname)
    for tick in bp[0].yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontname(fname)
    plt.ylabel('Violations', fontsize = fontsize + 10, fontname = fname)

    syst = '   Delphi   '
    yhook = {'1.1': -18.0, '1.2': -15.00, 'all': -18.0}
    #if str(slo) == '1.2': yhook += 2.0
    props = dict(boxstyle = 'round', facecolor = 'white', edgecolor = 'black', alpha = 0.5, linewidth = linewidth + 0.5)
    def get_x(df, position): return ((len(set(df.columns)) * position - 3) / 2)
    y_box = 0.2
    plt.text(get_x(df, 2), yhook[slo], syst, fontsize = fontsize - 2, ha='center', va = 'center', bbox = props, fontname = fname, zorder = 3)
    plt.text(get_x(df, 4), yhook[slo], syst, fontsize = fontsize - 2, ha='center', va = 'center', bbox = props, fontname = fname, zorder = 3)
    plt.text(get_x(df, 6), yhook[slo], syst, fontsize = fontsize - 2, ha='center', va = 'center', bbox = props, fontname = fname, zorder = 3)
    dboxl = FancyBboxPatch((get_x(df, 2) - 1.75, y_box), 3.6, 10, boxstyle="round,pad=0.2",fc = 'grey', ec = 'black', zorder = 2, alpha = 0.1)
    yoffset = 2.5
    dboxm = FancyBboxPatch((get_x(df, 4) - 1.75, y_box + yoffset), 3.6, 15.5 - 3 * int('1.2' in str(slo)), boxstyle="round,pad=0.2",fc = 'grey', ec = 'black', zorder = -1, alpha = 0.1)
    yoffset += 2.5 if '1.2' not in str(slo) else 0.0
    dboxh = FancyBboxPatch((get_x(df, 6) - 1.75, y_box + yoffset), 3.6, 15, boxstyle="round,pad=0.2",fc = 'grey', ec = 'black', zorder = -1, alpha = 0.1)
    for b in [dboxl, dboxm, dboxh]: bp[0].add_patch(b)
    
    y1, y2 = plt.ylim()
    bp[0].yaxis.set_label_coords(-0.05, 0.5)
    xanchor = 4.0#len(set(df.columns)) - 2.40
    yanch = {'1.1': 2.5, '1.2': 2.1, 'all': 2.5}
    yanchor = y2 - yanch[slo]
    y_pos = {'1.1': 0.38, '1.2': 0.38, 'all': 0.38}
    y_size = {'1.1': 2.0, '1.2': 1.8, 'all': 2.0}
    box_size = 0.5
    box_y = {'1.1': 1.42, '1.2': 1.2, 'all': 1.42}
    interval = 0.15
    rbox = FancyBboxPatch((xanchor, yanchor - y_pos[slo]), 12.25, (y_size[slo]), boxstyle="round,pad=0.3",fc = 'white', ec = 'black', zorder = 3)
    plt.text(xanchor, yanchor, 'Intensity:', fontsize = fontsize, fontname = fname, zorder = 4)
    bp[0].add_patch(rbox)
    xanchor += 3.5
    bp[0].add_patch(Rectangle((xanchor,yanchor), box_size, box_y[slo], facecolor = colors[0], edgecolor = edgecs[0], linewidth = linewidth, zorder = 4))
    plt.text(xanchor + box_size + interval, yanchor, 'Low', fontsize = fontsize, fontname = fname)
    xanchor += box_size + 2.0
    bp[0].add_patch(Rectangle((xanchor, yanchor), box_size, box_y[slo], facecolor = colors[len(set(df.columns))], edgecolor = edgecs[len(set(df.columns))], linewidth = linewidth, zorder = 4))
    plt.text(xanchor + box_size + interval, yanchor, 'Medium', fontsize = fontsize, fontname = fname)
    xanchor += box_size + 3.4
    bp[0].add_patch(Rectangle((xanchor, yanchor), box_size, box_y[slo], facecolor = colors[2 * len(set(df.columns))], edgecolor = edgecs[2 * len(set(df.columns))], linewidth = linewidth, zorder = 4))
    plt.text(xanchor + box_size + interval, yanchor, 'High', fontsize = fontsize, fontname = fname)

    #plt.tight_layout()
    plt.savefig(f"violations_{slo}.png")
    plt.clf()


In [237]:
for slo in ['1.1', '1.2', 'all']:
    violations_boxplot(slo)